#TASK 1

Function that returns all records based on a pattern (example of pattern: part of name, surname, phone number)

In [1]:
import psycopg2

conn = psycopg2.connect(
    host='ep-divine-fog-a55emvje-pooler.us-east-2.aws.neon.tech',
    database='neondb',
    user='neondb_owner',
    password='npg_Um02TyoiHXhw'
)
cur = conn.cursor()

cur.execute("DROP FUNCTION IF EXISTS search_user_paginated(TEXT);")

sql_function = """
CREATE FUNCTION search_user_paginated(
    pattern TEXT
) RETURNS TABLE (
    id INTEGER,
    name TEXT,
    surname TEXT,
    phone TEXT
) AS $$
BEGIN
    RETURN QUERY
    SELECT u.id, u.name, u.surname, u.phone
    FROM user_information u
    WHERE u.name ILIKE CONCAT('%', pattern, '%')
       OR u.surname ILIKE CONCAT('%', pattern, '%')
       OR u.phone ILIKE CONCAT('%', pattern, '%')
    ORDER BY u.id;
END;
$$ LANGUAGE plpgsql;
"""

cur.execute(sql_function)
conn.commit()

def search_user_information(pattern):
    cur.execute("SELECT * FROM search_user_paginated(%s);", (pattern,))
    return cur.fetchall()

pattern = input("Enter part of name, surname or phone: ")

results = search_user_information(pattern)

if results:
    print("Found:")
    for row in results:
        print(row)
else:
    print("Nothing found.")

cur.close()
conn.close()

Found:
(4, 'Zheka', 'Simon', '3332224444')
(6, 'Balkan', 'Kairat', '3332224444')


#TASK 2

Create procedure to insert new user by name and phone, update phone if user already exists

In [2]:
import psycopg2

conn = psycopg2.connect(
    host='ep-divine-fog-a55emvje-pooler.us-east-2.aws.neon.tech',
    database='neondb',
    user='neondb_owner',
    password='npg_Um02TyoiHXhw'
)
cur = conn.cursor()

cur.execute("DROP FUNCTION IF EXISTS insert_or_update_user(TEXT, TEXT);")
conn.commit()

cur.execute("""
CREATE OR REPLACE FUNCTION insert_or_update_user(
    name TEXT,
    phone TEXT
) RETURNS TEXT AS $$
BEGIN
    IF EXISTS (SELECT 1 FROM user_information WHERE user_information.name = $1) THEN
        UPDATE user_information 
        SET phone = $2 
        WHERE user_information.name = $1;
        RETURN 'Phone number updated';
    ELSE
        INSERT INTO user_information (name, phone) 
        VALUES ($1, $2);
        RETURN 'User added to the database';
    END IF;
END;
$$ LANGUAGE plpgsql;
""")
conn.commit()

def insert_or_update_user(name, phone):
    cur.execute("SELECT * FROM insert_or_update_user(%s, %s);", (name, phone))
    result = cur.fetchone()[0]
    print(result)
    conn.commit()

name_input = input("Enter user's name: ")
phone_input = input("Enter user's phone number: ")

insert_or_update_user(name_input, phone_input)

cur.close()
conn.close()

User added to the database


#TASK 3

Create procedure to insert many new users by list of name and phone. Use loop and if statement in stored procedure. Check correctness of phone in procedure and return all incorrect data.

In [ ]:
import psycopg2

conn = psycopg2.connect(
    host='ep-divine-fog-a55emvje-pooler.us-east-2.aws.neon.tech',
    database='neondb',
    user='neondb_owner',
    password='npg_Um02TyoiHXhw'
)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS user_information (
    id SERIAL PRIMARY KEY,
    name TEXT,
    surname TEXT,
    phone TEXT
);
""")
conn.commit()

cur.execute("DROP FUNCTION IF EXISTS insert_or_update_user(TEXT, TEXT, TEXT);")
conn.commit()

cur.execute("""
CREATE OR REPLACE FUNCTION insert_or_update_user(
    p_name TEXT,
    p_surname TEXT,
    p_phone TEXT
) RETURNS TEXT AS $$
BEGIN
    IF p_phone IS NULL OR NOT p_phone ~ '^\d{7,15}$' THEN
        RETURN 'Invalid phone number';
    END IF;

    IF EXISTS (SELECT 1 FROM user_information WHERE name = p_name AND surname = p_surname) THEN
        UPDATE user_information 
        SET phone = p_phone 
        WHERE name = p_name AND surname = p_surname;
        RETURN 'Phone number updated';
    ELSE
        INSERT INTO user_information (name, surname, phone) 
        VALUES (p_name, p_surname, p_phone);
        RETURN 'User added to the database';
    END IF;
END;
$$ LANGUAGE plpgsql;
""")
conn.commit()

names = input("Enter names (comma separated): ").split(',')
surnames = input("Enter surnames (comma separated): ").split(',')
phones = input("Enter phones (comma separated): ").split(',')

names = [n.strip() for n in names]
surnames = [s.strip() for s in surnames]
phones = [p.strip() for p in phones]

max_len = max(len(names), len(surnames), len(phones))
while len(names) < max_len:
    names.append('')
while len(surnames) < max_len:
    surnames.append('')
while len(phones) < max_len:
    phones.append('')

invalid_entries = []

for i in range(max_len):
    name = names[i]
    surname = surnames[i]
    phone = phones[i]

    cur.execute("SELECT * FROM insert_or_update_user(%s, %s, %s);", (name, surname, phone))
    result = cur.fetchone()[0]
    if result != 'Phone number updated' and result != 'User added to the database':
        invalid_entries.append(f"{name} {surname} ({phone})")

conn.commit()

if invalid_entries:
    print("⚠️ Invalid phone numbers found:")
    for entry in invalid_entries:
        print(" -", entry)
else:
    print("✅ All users were inserted or updated successfully.")

cur.close()
conn.close()

⚠️ Invalid phone numbers found:
 - tuk duk (bbc777)
 - kuk vuk (nnn667)


#TASK 4

Create function to querying data from the tables with pagination (by limit and offset)

In [32]:
import psycopg2

conn = psycopg2.connect(
    host='ep-divine-fog-a55emvje-pooler.us-east-2.aws.neon.tech',
    database='neondb',
    user='neondb_owner',
    password='npg_Um02TyoiHXhw'
)
cur = conn.cursor()

cur.execute("""
CREATE OR REPLACE FUNCTION get_users_paginated(
    page_limit INT,
    page_offset INT
) RETURNS TABLE (
    id INTEGER,
    name TEXT,
    surname TEXT,
    phone TEXT
) AS $$
BEGIN
    RETURN QUERY
    SELECT user_information.id, user_information.name, user_information.surname, user_information.phone
    FROM user_information
    ORDER BY user_information.id
    LIMIT page_limit OFFSET page_offset;
END;
$$ LANGUAGE plpgsql;
""")
conn.commit()

limit = int(input("Enter number of records per page (limit): "))
offset = int(input("Enter offset (skip how many records): "))

cur.execute("SELECT * FROM get_users_paginated(%s, %s);", (limit, offset))
rows = cur.fetchall()

if rows:
    for row in rows:
        print(f"ID: {row[0]}, Name: {row[1]}, Surname: {row[2]}, Phone: {row[3]}")
else:
    print("No records found.")

cur.close()
conn.close()


ID: 2, Name: Aibar, Surname: Smailov, Phone: 9876543210
ID: 3, Name: Askar, Surname: Kaldibayev, Phone: 77766655509
ID: 4, Name: Zheka, Surname: Simon, Phone: 3332224444
ID: 5, Name: Zelda, Surname: Duman, Phone: 12345677890
ID: 6, Name: Balkan, Surname: Kairat, Phone: 3332224444
ID: 7, Name: John, Surname: None, Phone: 1234567890
ID: 8, Name: Xxx, Surname: None, Phone: 123123123


#TASK 5

Implement procedure to deleting data from tables by username or phone

In [ ]:
import psycopg2

# Connect to the database
conn = psycopg2.connect(
    host='ep-divine-fog-a55emvje-pooler.us-east-2.aws.neon.tech',
    database='neondb',
    user='neondb_owner',
    password='npg_Um02TyoiHXhw'
)
cur = conn.cursor()

cur.execute("""
DROP PROCEDURE IF EXISTS delete_user_data(TEXT, TEXT, TEXT);
""")
conn.commit()

cur.execute("""
CREATE OR REPLACE PROCEDURE delete_user_data(
    IN p_name TEXT,
    IN p_surname TEXT,
    IN p_phone TEXT
)
LANGUAGE plpgsql
AS $$
BEGIN
    -- Delete by name and surname
    IF p_name IS NOT NULL AND p_surname IS NOT NULL THEN
        DELETE FROM user_information WHERE name = p_name AND surname = p_surname;
    -- Delete by phone
    ELSIF p_phone IS NOT NULL AND p_phone <> '' THEN
        DELETE FROM user_information WHERE phone = p_phone;
    ELSE
        RAISE EXCEPTION 'You must provide either both name and surname, or a phone number.';
    END IF;
END;
$$;
""")
conn.commit()
print("✅ Stored procedure created successfully.")

def delete_user(name=None, surname=None, phone=None):
    try:
        cur.execute("CALL delete_user_data(%s, %s, %s);", (name, surname, phone))
        conn.commit()
        print("✅ User data has been deleted successfully.")
    except Exception as e:
        print("❌ Error deleting user data:", e)
        conn.rollback()

print("Enter the user data you want to delete:")

name = input("Enter name (or leave blank if not deleting by name): ").strip() or None
surname = input("Enter surname (or leave blank if not deleting by surname): ").strip() or None
phone = input("Enter phone number (or leave blank if not deleting by phone): ").strip() or None

if (name and surname) or phone:
    delete_user(name, surname, phone)
else:
    print("❌ You must provide either both name and surname, or a phone number.")

cur.close()
conn.close()


✅ Stored procedure created successfully.
Enter the user data you want to delete:
✅ User data has been deleted successfully.
